In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [2]:
#Ler a Base 
df = pd.read_excel('base_consolidada.xlsx')
df.drop(columns='Unnamed: 0',inplace=True)

In [3]:
#O dataset que foi tratado na etapa de preparação mostra as depesas, anuais e mensais, de diversos senadores.

#O campo de interesse da análise é o "Valor Reembolsado", que expressa o quanto do valor da despesa o senador deixou de pagar
#e foi rembolsado através de dinheiro público, ou seja, o quanto essa despesa custou de fato.

df.head(10)

,ANO,MES,SENADOR,TIPO_DESPESA,FORNECEDOR,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,DIA,DT_DATA
0,2009,12,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",GILBERTO PISELO DO NASCIMENTO,11/12/2009,SEM DETALHAMENTO,5000.00,11,2009-12-11
1,2009,12,ACIR GURGACZ,Divulgação da atividade parlamentar,INTERCOM INTERMEDIAÇÕES E COMUNICAÇÃO INTEGRAD...,09/12/2009,SEM DETALHAMENTO,12620.00,9,2009-12-09
2,2009,4,ADELMIR SANTANA,"Locomoção, hospedagem, alimentação, combustíve...",Cascol combustíveis para veículos Ltda,12/04/2009,SEM DETALHAMENTO,179.01,12,2009-04-12
3,2009,4,ADELMIR SANTANA,"Locomoção, hospedagem, alimentação, combustíve...",Ribeiro e Pereira Ltda,04/04/2009,SEM DETALHAMENTO,30.00,4,2009-04-04
4,2009,4,ADELMIR SANTANA,"Locomoção, hospedagem, alimentação, combustíve...",Tudo de Bom Comércio de alimentos Ltda,14/04/2009,SEM DETALHAMENTO,215.80,14,2009-04-14
5,2009,5,ADELMIR SANTANA,"Contratação de consultorias, assessorias, pesq...",C A Guimarães Baião Consultores Associados S/S...,05/06/2009,SEM DETALHAMENTO,12475.00,5,2009-05-05
6,2009,5,ADELMIR SANTANA,"Locomoção, hospedagem, alimentação, combustíve...",ML alimentações e diversões S/A,02/05/2009,SEM DETALHAMENTO,420.00,2,2009-05-02
7,2009,5,ADELMIR SANTANA,"Locomoção, hospedagem, alimentação, combustíve...",Parigi - Restaurante fasano LTDA,21/05/2009,SEM DETALHAMENTO,546.00,21,2009-05-21
8,2009,6,ADELMIR SANTANA,"Locomoção, hospedagem, alimentação, combustíve...",A M Comercial de Combustíveis Ltda,24/06/2009,SEM DETALHAMENTO,130.00,24,2009-06-24
9,2009,6,ADELMIR SANTANA,"Locomoção, hospedagem, alimentação, combustíve...",Auto posto J B Ltda,24/06/2009,SEM DETALHAMENTO,250.00,24,2009-06-24


In [4]:
#De cara, vamos entender quais senadores, historicamente, são os mais custosos.

df_historico = df.groupby('SENADOR').sum()['VALOR_REEMBOLSADO']
df_historico = df_historico.sort_values()
df_historico = df_historico.round(2)

In [5]:
fig = px.bar(df_historico.tail(5),
            orientation = 'h',
            title='Quais Senadores mais caros?',
            labels=dict(SENADOR="Senador", value="Valor da Despesa (R$)"),
            text='value')

fig.update_layout(showlegend=False)

fig.show()

In [6]:
#Eduardo Amorim, você foi o escolhido. Vamos ver quais tipos de despesas favoritas do senador


df_em = df.loc[df['SENADOR'] == 'EDUARDO AMORIM']
df_em_dp = df_em.groupby('TIPO_DESPESA',as_index=False).sum()
df_em_dp['DESPESAS'] = 'DESPESAS'

In [7]:
fig = px.pie(df_em_dp, 
             values="VALOR_REEMBOLSADO", 
             names="TIPO_DESPESA",
            hole=.6)

fig.update_layout(legend=dict(
    orientation="h"
))

fig.show()

In [8]:
#E qual a empresa favorita do senador para voar?
df_em_fn = df_em.loc[df_em['TIPO_DESPESA'] == 'Passagens aéreas, aquáticas e terrestres nacionais']

#Aqui vou garantir a padronização dos dados de fornecedor
df_em_fn['FORNECEDOR'] = df_em_fn['FORNECEDOR'].str.upper()
df_em_fn['FORNECEDOR'] = df_em_fn['FORNECEDOR'].str.strip()
df_em_fn['VALOR_REEMBOLSADO'] = df_em_fn['VALOR_REEMBOLSADO'].round(2) 

df_em_fn = df_em_fn.groupby('FORNECEDOR',as_index=False).sum()

df_em_fn = df_em_fn.sort_values('VALOR_REEMBOLSADO')

df_em_fn.replace({'AZUL LINHAS AÉRAS BRASILEIRAS' : 'AZUL',
'GOL LINHAS AÉREAS INTELIGENTES': 'GOL'})

C:\Users\bruno\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bruno\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bruno\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

,FORNECEDOR,ANO,MES,VALOR_REEMBOLSADO,DIA
3,BR PETROBRAS,4027,7,233.07,51
5,GOL,2011,8,514.13,10
2,AZUL LINHAS AÉREAS BRASILEIRAS,2012,6,799.98,6
1,AZUL,4027,8,1977.74,25
8,TRIP,10058,48,4576.05,74
4,GOL,74475,250,35151.39,647
7,TAM,185175,579,98323.51,1504
0,AVIANCA,467038,1590,198166.26,3703
6,PROPAGTUR - PROPAG TURISMO LTDA,869117,3104,4439280.64,6786


In [9]:
fig = px.bar(df_em_fn.round(2),
            y='FORNECEDOR',
            x='VALOR_REEMBOLSADO',
            orientation = 'h',
            title='Os fornecedores do senador',
            labels=dict(FORNECEDOR="Fornecedores", VALOR_REEMBOLSADO="Valor da Despesa por Fornecedor (R$)"),
            text='VALOR_REEMBOLSADO')

fig.update_layout(showlegend=False)

fig.show()

In [52]:
#Ok,não dá pra analisar caso a caso... por enquanto. 
#No final, vou desponibilizar uma visualização para que você possa se divertir.

#Agora, vamos analisar os dados com um olhar um pouco mais estatistico

#Primeiro, vamos agregar os dados para análise
df_data = df.groupby(['ANO']).agg(['sum','mean'])
df_data = df_data['VALOR_REEMBOLSADO'].reset_index()
df_data.loc[df_data['ANO'] != 2022]
df_data = df_data.loc[df_data['ANO'] != 2022]


,ANO,sum,mean
0,2009,8916781.81,564.925355
1,2010,10742774.42,589.387964
2,2011,18981831.39,742.870671
3,2012,23312824.69,773.895389
4,2013,24586706.51,838.106985
5,2014,22479193.54,1023.223339
6,2015,28845605.67,1109.105109
7,2016,25200932.98,944.598110
8,2017,26513035.67,986.348053
9,2018,25486332.73,1044.436224


In [61]:
#Aqui vamos dar uma olhadinha em quanto do nosso dinheiro é reembolsado de forma anual, para nossos queridos senadores

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=df_data['ANO'], y=df_data['sum'],
                    mode='lines',
                    name='Despesas totais',
                    ),row=1, col=1)

fig.add_trace(go.Scatter(x=df_data['ANO'], y=df_data['mean'],
                    mode='lines+markers',
                    name='Média de despesas',
                    ),row=2, col=1)

fig.update_layout(title_text="Despesas Totais e Média de Despesas | Anuais")

fig.show()